In [42]:
%load_ext autoreload
%autoreload 2 
import sys
for p in sys.path:
    print(p)
!jupyter kernelspec list
!pip install sklearn
#!pip install networkx
#!pip install node2vec
import numpy as np
import math as math 
import tensorflow as tf
import graph_synthesis as gs
from random_mini_batches import random_mini_batches3
from random_mini_batches import random_mini_batches2
import networkx as nx
import node2vec as n2v
from sklearn.cluster import KMeans

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload

/home/ubuntu/src/cntk/bindings/python
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python36.zip
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/lib-dynload
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/IPython/extensions
/home/ubuntu/.ipython
Available kernels:
  python2           /home/ubuntu/.local/share/jupyter/kernels/python2
  pyspark3kernel    /home/ubuntu/anaconda3/envs/tensorflow_p36/share/jupyter/kernels/pyspark3kernel
  pysparkkernel     /home/ubuntu/anaconda3/envs/tensorflow_p36/share/jupyter/kernels/pysparkkernel
  python3           /home/ubuntu/anaconda3/envs/tensorflow_p36/share/jupyter/kernels/python3
  sparkkernel       /home/ubuntu/anaconda3/envs/tensorflow_p36/share/jupyter/kernels/sparkkernel
  sparkrkernel      /home/

In [43]:
class GCN:
    
    def __init__(self, inputdat, labeldat, vertexclass, vertexlabel, testdat, testlabeldat, testvertexclass, testvertexlabel, layer_dims, decayrate = 0.99,
                 l2reg = 1e-2, learning_rate = 0.008, batch_size = 128, epoch_nums = 3000):
        
        #Data
        self.inputdat = inputdat
        self.labeldat = labeldat
        self.vertexdat = vertexlabel
        self.testdat = testdat
        self.testlabeldat = testlabeldat
        self.testvertexdat = testvertexlabel
        self.num_eg = inputdat.shape[0]
        self.inputdim = inputdat.shape[1]
        self.vertexdim = vertexlabel.shape[2]
        self.vertexclass = vertexclass
        self.testvertexclass = testvertexclass
        
        #Hyperparameters
        self.layer_dims = layer_dims
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.epoch_nums = epoch_nums
        self.l2reg = l2reg  
        self.decayrate = decayrate 
        
        #Initialize
        tf.reset_default_graph()
        self.initdata()
        
        #Create graph
        self.output, self.gcn = self.model()
        self.optimizer, self.cost = self.train()
        self.prediction, self.accuracy = self.test()
        self.summary_op = self.create_summaries()
        
        #Run Tensorflow
        config = tf.ConfigProto(allow_soft_placement = True, log_device_placement = True, device_count = {'GPU': 1})
        self.sess = tf.Session(config = config)
        init = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
        self.sess.run(init)
        
        #Tensorboard setup
        self.writer = tf.summary.FileWriter('./CNNSave',  (self.sess).graph)
        
        #Train model 
        self.graph_classifier()
        self.prediction_num, self.error_num = self.evaluate("train") 
        #self.evaluate_cluster()
    def  create_summaries(self):
        with tf.name_scope("summaries"):
            tf.summary.scalar("loss", self.cost)
            #tf.summary.scalar("output", self.output)
            tf.summary.scalar("accuracy", self.accuracy)
            tf.summary.histogram("histogram_loss", self.cost)
            #tf.summary.histogram("histogram_loss", self.output)
            summary_op = tf.summary.merge_all()
        return summary_op
    
    def initdata(self):
        self.tflearnrate = tf.placeholder(tf.float32, name = "learnrate")
        self.mode = tf.placeholder(tf.bool, name = "mode")
        self.input = tf.placeholder(tf.float32, shape = [None, self.inputdim, self.inputdim], name = "input")
        self.vertexlabel = tf.placeholder(tf.float32, shape = [None, self.inputdim, self.vertexdim], name = "vertex")
        self.label = tf.placeholder(tf.float32, shape = [None, 1], name = "label")

    
    def model(self):
        layer_dims = self.layer_dims
        layers = {}
        
        inputA = self.input
        inputX = self.vertexlabel
        
        scopegcn = 'GCN'
        with tf.variable_scope(scopegcn):
            #GCN 1 
            #name = 'GCN' + str(1)
            gcn = self.graph_convolutional_layer(inputX, inputA, scopegcn, 20)
        scope = 'batchnorm'+str(0)
        with tf.variable_scope(scope):
            gcn = tf.contrib.layers.batch_norm(gcn, is_training = self.mode, scope = 'batchnorm')
        for i in range(4):
            with tf.variable_scope(scopegcn, reuse = True):
                gcn = self.graph_convolutional_layer(gcn, inputA, scopegcn, 20)
            scope = 'batchnorm'+str(i+1)
            with tf.variable_scope(scope):
                gcn = tf.contrib.layers.batch_norm(gcn, is_training = self.mode, scope = 'batchnorm')
            
        """#GCN 1 
        name = 'GCN' + str(1)
        gcn1 = self.graph_convolutional_layer(inputX, inputA, name, 15) 
        
        #GCN 2 
        name = 'GCN' + str(2)
        gcn2 = self.graph_convolutional_layer(gcn1, inputA, name, 10) 
        
        #GCN 3
        name = 'GCN' + str(3)
        gcn3 = self.graph_convolutional_layer(gcn2, inputA, name, 5)"""
        
        size = gcn.shape[1]
        #print(size)
        pool_flat = gcn[:,size-1,:]
        #pool_flat = tf.reshape(tf.reduce_mean(gcn, axis = 2), [-1, size])
    
        layers[0] = pool_flat
        #Dense layers
        for i in range(len(layer_dims)-1):
            if (i < len(layer_dims) - 2):
                scope = 'denselayers' + str(i+1)
                with tf.variable_scope(scope):
                    layers[i+1] = tf.contrib.layers.fully_connected(num_outputs = layer_dims[i+1], \
                                                        activation_fn = None, inputs = layers[i], scope = 'dense')
                    layers[i+1] = tf.contrib.layers.batch_norm(layers[i+1], is_training = self.mode, scope = 'batchnorm')
                    layers[i+1] = tf.nn.relu(layers[i+1], 'relu')
            else:
                scope = 'denselayers' + str(i+1)
                with tf.variable_scope(scope):
                    layers[len(layer_dims)-1] = tf.layers.dense(units = layer_dims[len(layer_dims)-1], \
                                                    activation = tf.nn.sigmoid, inputs = layers[len(layer_dims)-2])
        
        output = layers[len(layer_dims)-1]
        return output, gcn  
    
    def train(self):
        #Saver for data 
        self.saver = tf.train.Saver()
        self.global_step = tf.Variable(0, dtype = tf.int32, trainable = False, name = 'global_step')
        
        #Define cost 
        gap_loss = tf.reduce_max(self.output) - tf.reduce_min(self.output)
        reg_loss = self.l2reg * sum(tf.nn.l2_loss(tf_var) for tf_var in tf.trainable_variables()) 
        cost = (tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = self.output, labels = self.label)) + \
                                                                                            reg_loss)
        #cost = (tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = self.output, labels = self.label)) + \
                                                                                            #reg_loss) * 1/(0.01+gap_loss)
        
        #Batch Normalization
        update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
        with tf.control_dependencies(update_ops):
            optimizer = tf.train.AdamOptimizer(learning_rate=self.tflearnrate).minimize(cost, global_step = self.global_step)
            
        return optimizer, cost 
    
    def test(self):
        prediction = tf.greater(self.output, tf.constant(0.5, dtype = tf.float32))
        prediction = tf.cast(prediction, dtype = tf.float32)
        with tf.name_scope('accuracy'):
            with tf.name_scope('correct_prediction'):
                correct_prediction = tf.equal(tf.cast(prediction, dtype = tf.float32), self.label)
            with tf.name_scope('accuracy'):
                accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
                tf.summary.scalar('accuracy', accuracy)
        return prediction, accuracy
    
    def graph_classifier(self):
        curlearnrate = self.learning_rate
        step = 0 
        for epochs in range(self.epoch_nums):
            output = []
            mini_batch = random_mini_batches3(self.inputdat, self.labeldat, self.vertexdat, self.batch_size)
            curlearnrate = curlearnrate * self.decayrate 
            for dat in mini_batch:
                step = step + 1 
                _, cost, output, summary = self.sess.run([self.optimizer, self.cost, self.output, self.summary_op], 
                                                feed_dict = {self.input: dat[0], self.label: dat[1], self.vertexlabel: dat[2],\
                                                             self.mode: True, self.tflearnrate: curlearnrate})
                (self.writer).add_summary(summary, global_step = step)
            if ((epochs+1) % 50 == 0):
                
                _, errortrain = self.evaluate("train")
                output, cost, accuracy = self.sess.run([self.output, self.cost, self.accuracy], \
                        feed_dict = {self.input: self.inputdat, self.label: self.labeldat, self.vertexlabel: self.vertexdat, \
                                     self.mode: False, self.tflearnrate: curlearnrate})
                (self.saver).save(self.sess, 'checkpoints/CNN', global_step=self.global_step)
                _, errortest = self.evaluate("test")
                print("epoch", epochs + 1, "    |    ", "cost", "%.6e" % cost, "    |    ", 
                      "error train", "%.3f" % (errortrain * 100), "    |    ", "error test", "%.3f" % (errortest * 100), 
                      "   |   ", "output gap", "%.6e" % (np.max(output) - np.min(output)))
            
                #if ((epochs+1) % 200 == 0):
                #    print(output[0], output[len(output)-1])
                
    def evaluate(self, mode):
        #Get predictions
        
        if mode == "train":
            predictions = self.sess.run([self.prediction], 
                                    feed_dict = {self.input: self.inputdat, self.label: self.labeldat, self.vertexlabel: self.vertexdat,\
                                                 self.mode: False})
            predictions = np.array(predictions).reshape((self.num_eg,1))
            #predictions = predictions.astype(int)
        
            #Compute error
            error = np.sum((predictions - self.labeldat)**2)/self.num_eg
            #print("error " + mode, error * 100)
        else:
            predictions, gcn = self.sess.run([self.prediction, self.gcn], 
                                    feed_dict = {self.input: self.testdat, self.label: self.testlabeldat, self.vertexlabel: self.testvertexdat,\
                                                 self.mode: False})
            predictions = np.array(predictions).reshape((self.num_eg,1))
            #predictions = predictions.astype(int)
        
            #Compute error
            error = np.sum((predictions - self.testlabeldat)**2)/self.num_eg
            #print("error " + mode, error * 100)
            
        return predictions, error
    
    def evaluate_cluster(self):
        #Get predictions
        
        predictions, gcn = self.sess.run([self.prediction, self.gcn], 
                                    feed_dict = {self.input: self.testdat, self.label: self.testlabeldat, self.vertexlabel: self.testvertexdat,\
                                                 self.mode: False})
        print(gcn[0,0:self.inputdim,:])
        
        for i in range(10):
            graph = gcn[i,0:self.inputdim,:]
            label = self.vertexclass[i,:]
            kmeans = KMeans(n_clusters=2, random_state=0).fit(graph)
            print(label)
            print(kmeans)
        
        return None
    
    def graph_convolutional_layer(self, X, An, name, dim):
        
        AX = tf.matmul(An, X)
        btsize = tf.shape(AX)[0]
        vertex = AX.shape[2]
        
        #AX = tf.tile(tf.expand_dims(AX, 1), [1, rep, 1, 1])
        
        #print(btsize)
        #print(vertex)
        #with tf.variable_scope(name):
        W = tf.get_variable('W', [vertex, dim])
        mW = tf.tile(tf.expand_dims(W, 0), [btsize, 1, 1]) 
        
        output = tf.matmul(AX,mW)
        
        output = tf.nn.relu(output)
        #print(output.shape)
        #print(output.shape)
        return output 
    
    def gated_graph_convolutional_layer(self, X, An, name, dim):
        
        AX = tf.matmul(An, X)
        btsize = tf.shape(AX)[0]
        numver = AX.shape[1]
        vertex = AX.shape[2]
        
        #AX = tf.tile(tf.expand_dims(AX, 1), [1, rep, 1, 1])
        
        #print(btsize)
        #print(vertex)
        #with tf.variable_scope(name):
        W = tf.get_variable('W', [vertex, dim])
        mW = tf.tile(tf.expand_dims(W, 0), [btsize, 1, 1]) 
        
        Wn = tf.get_variable('Wn', [vertex, dim])
        mWn = tf.tile(tf.expand_dims(Wn, 0), [btsize, 1, 1]) 
        
        #bW = tf.get_variable('bW', [numver, dim])
        #mbW = tf.tile(tf.expand_dims(bW, 0), [btsize, 1, 1]) 
        
        output = tf.matmul(AX,mW) + tf.matmul(X,mWn) 
        
        output = tf.nn.relu(output)
        
        G = tf.get_variable('G', [vertex, dim])
        mG = tf.tile(tf.expand_dims(G, 0), [btsize, 1, 1]) 
        
        Gn = tf.get_variable('Gn', [vertex, dim])
        mGn = tf.tile(tf.expand_dims(Gn, 0), [btsize, 1, 1]) 
        
        #bG = tf.get_variable('bG', [numver, dim])
        #mbG = tf.tile(tf.expand_dims(bG, 0), [btsize, 1, 1]) 
        
        gate = tf.matmul(AX,mG) + tf.matmul(X,mGn)  
        
        gate = tf.nn.sigmoid(gate)
        
        output = tf.multiply((1-gate), X) + tf.multiply(gate, output)
        #print(output.shape)
        #print(output.shape)
        return output 
    
    #def graph_local_convolution(self, X):
    #    vertex = self.inputdim
        
    #    for i in range(vertex):
            
    #    W = tf.get_variable(name + 'W', [vertex, dim])
    #    mW = tf.tile(tf.expand_dims(W, 0), [btsize, 1, 1]) 
    #    return None"""

In [44]:
if  __name__ == "__main__":
    
    k=2
    num_eg = 2000
    vertex = 37
    vertexdim = 20 
    inputdim = vertexdim
    
    #train ensemble 
    #data = gs.generate_ensemble([int(num_eg/2), int(num_eg/2)], vertex, [1/3, 1/2])
    data = gs.generate_ensemble_v2_label(num_eg, vertex-1)
    #data = gs.generate_ensemble_same_label(num_eg, vertex-1)
    
    
    #Normalize input:
    datagraph = data["graphs"]
    for i in range(num_eg):
        curgraph = np.linalg.matrix_power(datagraph[i,:,:],k) + np.eye(vertex-1)
        #degree = np.sum(curgraph, axis = 0)
        #updategraph = np.minimum(curgraph, np.ones((vertex-1,vertex-1)))
        data["graphs"][i,:,:] = curgraph 
    
    data = gs.augment(data)
    for i in range(num_eg):
        curgraph = data["graphs"][i,:,:]
        degree = np.sum(curgraph, axis = 0)
        updategraph = 1/np.sqrt(degree) * curgraph * 1/np.sqrt(degree)
        data["graphs"][i,:,:] = updategraph
    datavertex = np.random.normal(loc=1, scale=0.0, size=(num_eg, vertex, vertexdim)).astype(np.float32)
    print(data["graphs"][1,:,:])
    #data = gs.generate_ensemble_n2v(num_eg, vertex)
    #test ensemble
    testdata = gs.generate_ensemble_v2_label(num_eg, vertex-1)
    #testdata = gs.generate_ensemble([int(num_eg/2), int(num_eg/2)], vertex, [1/3, 1/2])
    #testdata = gs.generate_ensemble_same_label(num_eg, vertex-1)
    
    
    testdatagraph = testdata["graphs"]
    for i in range(num_eg):
        curgraph = np.linalg.matrix_power(testdatagraph[i,:,:],2) + np.eye(vertex-1)
        #degree = np.sum(curgraph, axis = 0)
        #updategraph = np.minimum(curgraph, np.ones((vertex-1,vertex-1)))
        testdata["graphs"][i,:,:] = curgraph 
    testdata = gs.augment(testdata)
    for i in range(num_eg):
        curgraph = testdata["graphs"][i,:,:]
        degree = np.sum(curgraph, axis = 0)
        updategraph = 1/np.sqrt(degree) * curgraph * 1/np.sqrt(degree)
        testdata["graphs"][i,:,:] = updategraph 
        
    testdatavertex = np.random.normal(loc=1, scale=0.0, size=(num_eg, vertex, vertexdim)).astype(np.float32)
    #testdata = gs.generate_ensemble_n2v(num_eg, vertex)
    #reformat 
    data["graphs"] = data["graphs"].reshape((num_eg, vertex, vertex))
    testdata["graphs"] = testdata["graphs"].reshape((num_eg, vertex, vertex))
    
    #Fully connected model 
    GCN = GCN(data["graphs"], data["labels"], data["vertex"], datavertex, testdata["graphs"], testdata["labels"], testdata["vertex"], testdatavertex, [inputdim,15,10,5,1])
    
    #Test Model 
    #FullNN.inputdat = testdata["graphs"]
    #FullNN.labeldat = testdata["labels"]
    #FullNN.trainmode = False
    #FullNN.evaluate()

[[0.05571031 0.03676471 0.02631579 ... 0.02392344 0.03082192 0.02702703]
 [0.02785515 0.05514706 0.02255639 ... 0.02392344 0.0239726  0.02702703]
 [0.01949861 0.02205882 0.05639098 ... 0.0215311  0.0239726  0.02702703]
 ...
 [0.02785515 0.03676471 0.03383459 ... 0.05502392 0.03082192 0.02702703]
 [0.02506964 0.02573529 0.02631579 ... 0.0215311  0.05821918 0.02702703]
 [0.00278552 0.00367647 0.0037594  ... 0.00239234 0.00342466 0.02702703]]
epoch 50     |     cost 7.344568e-01     |     error train 50.000     |     error test 50.000    |    output gap 5.170703e-06
epoch 100     |     cost 7.297815e-01     |     error train 50.000     |     error test 50.000    |    output gap 0.000000e+00
epoch 150     |     cost 7.275183e-01     |     error train 50.000     |     error test 50.000    |    output gap 2.289861e-04
epoch 200     |     cost 7.258125e-01     |     error train 50.000     |     error test 50.000    |    output gap 9.187162e-04
epoch 250     |     cost 7.248008e-01     |     e

In [45]:
predictions, gcn = GCN.sess.run([GCN.prediction, GCN.gcn], 
                                    feed_dict = {GCN.input: GCN.testdat, GCN.label: GCN.testlabeldat, GCN.vertexlabel: GCN.testvertexdat,\
                                                 GCN.mode: False})
#print(predictions)
#print(gcn[0,0:GCN.inputdim,:])
cumerror = 0 
for i in range(500):
    graph = gcn[i,0:(GCN.inputdim-1),:]
    label = GCN.vertexclass[i,:]
    kmeans = KMeans(n_clusters=2).fit_predict(graph)
    label = label.reshape(int(label.shape[0]))
    #print(kmeans)
    idx0 = (kmeans == 0)
    idx1 = (kmeans == 1)
    label0 = label[idx0]
    label1 = label[idx1]
    error = 0 
    if (np.sum(label0) < len(label0)/2):
        error =  np.sum(label0)/len(label) + (len(label1) - np.sum(label1))/len(label)
    else:
        error = (len(label0) - np.sum(label0))/len(label) + np.sum(label1)/len(label)
    print(error)
    cumerror += error
print(cumerror / 500)

0.4444444444444444
0.5
0.41666666666666663
0.33333333333333337
0.4444444444444444
0.4444444444444445
0.4166666666666667
0.5
0.3888888888888889
0.4444444444444445
0.4444444444444444
0.4722222222222222
0.38888888888888884
0.4444444444444444
0.5
0.41666666666666663
0.4444444444444444
0.3888888888888889
0.3611111111111111
0.4722222222222222
0.4722222222222222
0.4722222222222222
0.5
0.4722222222222222
0.4444444444444445
0.4722222222222222
0.4722222222222222
0.4722222222222222
0.4444444444444444
0.3611111111111111
0.4444444444444444
0.4722222222222222
0.4722222222222222
0.4166666666666667
0.4444444444444444
0.4444444444444445
0.4166666666666667
0.4444444444444445
0.5
0.3333333333333333
0.4444444444444445
0.4166666666666667
0.3611111111111111
0.5
0.41666666666666663
0.5
0.5
0.41666666666666663
0.4722222222222222
0.38888888888888884
0.5
0.4722222222222222
0.4444444444444444
0.4722222222222222
0.4444444444444445
0.4444444444444444
0.4444444444444444
0.41666666666666663
0.4722222222222222
0.4722

0.4444444444444444
0.5
0.4722222222222222
0.4722222222222222
0.4722222222222222
0.5
0.5
0.3611111111111111
0.4444444444444445
0.4444444444444444
0.4722222222222222
0.4722222222222222
0.5
0.41666666666666663
0.4444444444444444
0.4166666666666667
0.41666666666666663
0.4385000000000014


In [ ]:
class FullNN:
    
    def __init__(self, inputdat, labeldat, testdat, testlabeldat, layer_dims, decayrate = 0.99,
                 l2reg = 1e-2, learning_rate = 0.008, batch_size = 128, epoch_nums = 15000):
        
        #Data
        self.inputdat = inputdat
        self.labeldat = labeldat
        self.testdat = testdat
        self.testlabeldat = testlabeldat
        self.num_eg = inputdat.shape[0]
        
        #Hyperparameters
        self.layer_dims = layer_dims
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.epoch_nums = epoch_nums
        self.l2reg = l2reg  
        self.decayrate = decayrate 
        
        #Initialize
        tf.reset_default_graph()
        self.initdata()
        
        #Create graph
        self.output = self.model()
        self.optimizer, self.cost = self.train()
        self.prediction, self.accuracy = self.test()
        self.summary_op = self.create_summaries()
        
        #Run Tensorflow
        config = tf.ConfigProto(allow_soft_placement = True, log_device_placement = True, device_count = {'GPU': 1})
        self.sess = tf.Session(config = config)
        init = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
        self.sess.run(init)
        
        #Tensorboard setup
        self.writer = tf.summary.FileWriter('./FullNNSave',  (self.sess).graph)
        
        #Train model 
        self.graph_classifier()
        self.prediction_num, self.error_num = self.evaluate("train") 
        
    def  create_summaries(self):
        with tf.name_scope("summaries"):
            tf.summary.scalar("loss", self.cost)
            tf.summary.scalar("accuracy", self.accuracy)
            tf.summary.histogram("histogram_loss", self.cost)
            summary_op = tf.summary.merge_all()
        return summary_op
    
    def initdata(self):
        self.tflearnrate = tf.placeholder(tf.float32, name = "learnrate")
        self.mode = tf.placeholder(tf.bool, name = "mode")
        self.input = tf.placeholder(tf.float32, shape = [None, self.layer_dims[0]], name = "input")
        self.label = tf.placeholder(tf.float32, shape = [None, 1], name = "label")

    
    def model(self):
        layer_dims = self.layer_dims
        layers = {}
        layers[0] = self.input
        for i in range(len(layer_dims)-1):
            if (i < len(layer_dims) - 2):
                scope = 'layers' + str(i+1)
                with tf.variable_scope(scope):
                    #layers[i+1] = tf.layers.dense(units = layer_dims[i+1], activation = tf.nn.relu, inputs = layers[i])
                    layers[i+1] = tf.contrib.layers.fully_connected(num_outputs = layer_dims[i+1], \
                                                        activation_fn = None, inputs = layers[i], scope = 'dense')
                    layers[i+1] = tf.contrib.layers.batch_norm(layers[i+1], is_training = self.mode, scope = 'batchnorm')
                    layers[i+1] = tf.nn.relu(layers[i+1], 'relu')
            else:
                scope = 'layers' + str(i+1)
                with tf.variable_scope(scope):
                    layers[len(layer_dims)-1] = tf.layers.dense(units = layer_dims[len(layer_dims)-1], \
                                                    activation = tf.nn.sigmoid, inputs = layers[len(layer_dims)-2])
        
        output = layers[len(layer_dims)-1]
        return output 
    
    def train(self):
        #Saver for data 
        self.saver = tf.train.Saver()
        self.global_step = tf.Variable(0, dtype = tf.int32, trainable = False, name = 'global_step')
        
        #Define cost 
        reg_loss = self.l2reg * sum(tf.nn.l2_loss(tf_var) for tf_var in tf.trainable_variables()) 
        cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = self.output, labels = self.label)) + \
                                                                                                            reg_loss 
        
        #Batch Normalization
        update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
        with tf.control_dependencies(update_ops):
            optimizer = tf.train.AdamOptimizer(learning_rate=self.tflearnrate).minimize(cost, global_step = self.global_step)
            
        return optimizer, cost 
    
    def test(self):
        prediction = tf.greater(self.output, tf.constant(0.5, dtype = tf.float32))
        with tf.name_scope('accuracy'):
            with tf.name_scope('correct_prediction'):
                correct_prediction = tf.equal(tf.cast(prediction, dtype = tf.float32), self.label)
            with tf.name_scope('accuracy'):
                accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
                tf.summary.scalar('accuracy', accuracy)
        return prediction, accuracy
    
    def graph_classifier(self):
        curlearnrate = self.learning_rate
        step = 0 
        for epochs in range(self.epoch_nums):
            output = []
            mini_batch = random_mini_batches2(self.inputdat, self.labeldat, self.batch_size)
            curlearnrate = curlearnrate * self.decayrate 
            for dat in mini_batch:
                step = step + 1 
                _, cost, output, summary = self.sess.run([self.optimizer, self.cost, self.output, self.summary_op], 
                                                feed_dict = {self.input: dat[0], self.label: dat[1], \
                                                             self.mode: True, self.tflearnrate: curlearnrate})
                (self.writer).add_summary(summary, global_step = step)
            if (epochs % 100 == 0):
                _, errortrain = self.evaluate("train")
                cost, accuracy = self.sess.run([self.cost, self.accuracy], \
                        feed_dict = {self.input: self.inputdat, self.label: self.labeldat, \
                                     self.mode: False, self.tflearnrate: curlearnrate})
                (self.saver).save(self.sess, 'checkpoints/CNN', global_step=self.global_step)
                _, errortest = self.evaluate("test")
                print("epoch", epochs, "    |    ", "cost", "%.6e" % cost, "    |    ", 
                      "error train", "%.3f" % (errortrain * 100), "    |    ", "error test", "%.3f" % (errortest * 100))
        
    def evaluate(self, mode):
        #Get predictions
        if mode == "train":
            predictions = self.sess.run([self.prediction], 
                                    feed_dict = {self.input: self.inputdat, self.label: self.labeldat, \
                                                 self.mode: False})
        else:
            predictions = self.sess.run([self.prediction], 
                                    feed_dict = {self.input: self.testdat, self.label: self.testlabeldat, \
                                                 self.mode: False})
        predictions = np.array(predictions).reshape((self.num_eg,1))
        predictions = predictions.astype(int)
        
        #Compute error
        error = np.sum((predictions - self.labeldat)**2)/self.num_eg
        #print("error " + mode, error * 100)
        
        return predictions, error

In [ ]:
num_eg = 4000
vertex = 37

inputdim = 20

#train ensemble 
#data = gs.generate_ensemble([int(num_eg/2), int(num_eg/2)], vertex, [1/3, 1/2])
data = gs.generate_ensemble_v2_label(num_eg, vertex-1)
#data = gs.generate_ensemble_same_label(num_eg, vertex-1)


#Normalize input:
datagraph = data["graphs"]
for i in range(num_eg):
    curgraph = np.linalg.matrix_power(datagraph[i,:,:],k) + np.eye(vertex-1)
    #degree = np.sum(curgraph, axis = 0)
    #updategraph = np.minimum(curgraph, np.ones((vertex-1,vertex-1)))
    data["graphs"][i,:,:] = curgraph 

data = gs.augment(data)
for i in range(num_eg):
    curgraph = data["graphs"][i,:,:]
    degree = np.sum(curgraph, axis = 0)
    updategraph = 1/np.sqrt(degree) * curgraph * 1/np.sqrt(degree)
    data["graphs"][i,:,:] = updategraph
datavertex = np.random.normal(loc=1, scale=0.0, size=(num_eg, vertex, vertexdim)).astype(np.float32)
print(data["graphs"][1,:,:])
#data = gs.generate_ensemble_n2v(num_eg, vertex)
#test ensemble
testdata = gs.generate_ensemble_v2_label(num_eg, vertex-1)
#testdata = gs.generate_ensemble([int(num_eg/2), int(num_eg/2)], vertex, [1/3, 1/2])
#testdata = gs.generate_ensemble_same_label(num_eg, vertex-1)


testdatagraph = testdata["graphs"]
for i in range(num_eg):
    curgraph = np.linalg.matrix_power(testdatagraph[i,:,:],2) + np.eye(vertex-1)
    #degree = np.sum(curgraph, axis = 0)
    #updategraph = np.minimum(curgraph, np.ones((vertex-1,vertex-1)))
    testdata["graphs"][i,:,:] = curgraph 
testdata = gs.augment(testdata)
for i in range(num_eg):
    curgraph = testdata["graphs"][i,:,:]
    degree = np.sum(curgraph, axis = 0)
    updategraph = 1/np.sqrt(degree) * curgraph * 1/np.sqrt(degree)
    testdata["graphs"][i,:,:] = updategraph 

testdatavertex = np.random.normal(loc=1, scale=0.0, size=(num_eg, vertex, vertexdim)).astype(np.float32)
#testdata = gs.generate_ensemble_n2v(num_eg, vertex)
#reformat 
data["graphs"] = data["graphs"].reshape((num_eg, vertex, vertex))
testdata["graphs"] = testdata["graphs"].reshape((num_eg, vertex, vertex))
    
#reformat 
GCN.testdat = data["graphs"]
GCN.testlabeldat = data["labels"]
GCN.testvertexdat = datavertex
gcndata = GCN.sess.run([GCN.gcn], feed_dict = {GCN.input: GCN.testdat, GCN.label: GCN.testlabeldat, GCN.vertexlabel: GCN.testvertexdat,\
                                                 GCN.mode: False})
gcndata = np.asarray(gcndata)
print(gcndata[:,0,vertex-1,:])


data["graphs"] = gcndata[:,:,vertex-1,:].reshape(num_eg, -1)

GCN.testdat = testdata["graphs"]
GCN.testlabeldat = testdata["labels"]
GCN.testertexdat = testdatavertex
gcntestdata = GCN.sess.run([GCN.gcn], feed_dict = {GCN.input: GCN.testdat, GCN.label: GCN.testlabeldat, GCN.vertexlabel: GCN.testvertexdat,\
                                                 GCN.mode: False})
gcntestdata = np.asarray(gcntestdata)
testdata["graphs"] = gcntestdata[:,:,(vertex-1),:].reshape((num_eg, -1))



#Fully connected model 
FullNN = FullNN(data["graphs"], data["labels"], testdata["graphs"], testdata["labels"], [inputdim,15,10,5,1])

#Test Model 
#FullNN.inputdat = testdata["graphs"]
#FullNN.labeldat = testdata["labels"]
#FullNN.trainmode = False
#FullNN.evaluate()

[[0.05639098 0.02364066 0.01111111 ... 0.02453988 0.01644737 0.02702703]
 [0.03759398 0.05673759 0.03333333 ... 0.04294479 0.02631579 0.02702703]
 [0.0075188  0.0141844  0.05555556 ... 0.01840491 0.01644737 0.02702703]
 ...
 [0.03007519 0.03309693 0.03333333 ... 0.05521472 0.02302632 0.02702703]
 [0.01879699 0.01891253 0.02777778 ... 0.02147239 0.05263158 0.02702703]
 [0.0037594  0.00236407 0.00555556 ... 0.00306748 0.00328947 0.02702703]]
[[-6.94824159e-02  4.51461971e-03  3.49673256e-03  2.12956965e-02
   9.27078128e-02 -1.39359817e-01  1.26771256e-02 -8.46166586e-07
  -5.39693795e-03  2.64952779e-02 -1.78540036e-01 -4.33678553e-02
   9.68384445e-02  2.96154022e-02 -5.10152727e-02 -3.21727101e-04
   9.60365832e-02  1.53666735e-02 -1.39003005e-02 -4.01819088e-02]]
epoch 0     |     cost 7.934240e-01     |     error train 50.000     |     error test 50.000
epoch 100     |     cost 7.095647e-01     |     error train 50.000     |     error test 50.000
epoch 200     |     cost 5.820081e-0

epoch 7900     |     cost 5.670444e-01     |     error train 5.275     |     error test 5.175
epoch 8000     |     cost 5.670366e-01     |     error train 5.275     |     error test 5.175
epoch 8100     |     cost 5.670445e-01     |     error train 5.250     |     error test 5.100
epoch 8200     |     cost 5.670369e-01     |     error train 5.275     |     error test 5.175
epoch 8300     |     cost 5.670371e-01     |     error train 5.275     |     error test 5.175
epoch 8400     |     cost 5.670413e-01     |     error train 5.250     |     error test 5.175
epoch 8500     |     cost 5.670332e-01     |     error train 5.250     |     error test 5.175
epoch 8600     |     cost 5.670383e-01     |     error train 5.275     |     error test 5.175
epoch 8700     |     cost 5.670311e-01     |     error train 5.275     |     error test 5.175
epoch 8800     |     cost 5.670320e-01     |     error train 5.250     |     error test 5.175
epoch 8900     |     cost 5.670422e-01     |     error train

In [ ]:
print(data["graphs"][i,:,:])